In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (30, 30)

import wikipedia
import torch
from torch import nn
import pickle
import pandas as pd
import numpy as np

import os
import more_itertools as mit
from copy import deepcopy
from IPython.core.display import display, HTML
from itertools import groupby

# load data

In [ ]:
with open('/mnt/efs/disambiguation_data/ix_to_token.pkl', 'rb') as f:
    ix_to_token = pickle.load(f)
    
token_to_ix = {token: ix for ix, token in ix_to_token.items()}

In [ ]:
ix = np.random.choice(100)
base_path = '/mnt/efs/disambiguation_data/{}/{}.pt'

tokens = torch.load(base_path.format('tokens', ix), map_location='cpu').numpy()
predictions = torch.load(base_path.format('predictions', ix), map_location='cpu').numpy()

# do science

In [ ]:
index = np.random.choice(128)
to_keep = tokens[index] != 91282

In [ ]:
token_seq = [ix_to_token[ix] for ix in tokens[index] if ix != token_to_ix['xxpad']]
' '.join(token_seq)

In [ ]:
val_seq = predictions[index][1][to_keep]

In [ ]:
#pd.Series(val_seq).plot.bar();

In [ ]:
prev_seq = np.concatenate([[0], val_seq[:-1]])
next_seq = np.concatenate([val_seq[1:], [0]])

In [ ]:
indexes_to_include = []

In [ ]:
standard_threshold = 0
lower_threshold = -0.5

for i in range(len(val_seq)):
    p, v, n = prev_seq[i], val_seq[i], next_seq[i]
    to_include = (
        (v > standard_threshold) or 
        ((v > lower_threshold) and 
         ((p > standard_threshold) or
          (n > standard_threshold)))
    )
    #print('\t'.join([str(round(p, 2)), 
    #                 str(round(v, 2)), 
    #                 str(round(n, 2)), 
    #                 str(to_include)]))
    
    indexes_to_include.append(to_include)
    

In [ ]:
groups = [list(group) for group in mit.consecutive_groups(np.where(indexes_to_include)[0])]
bool_groups = [list(group) for _, group in groupby(indexes_to_include)]

In [ ]:
links = []

for group in groups:
    try:
        text = ' '.join([token_seq[g] for g in group])
        url = wikipedia.page(text).url
        links.append(f'<a href="{url}">{text}</a>')

    except:
        links.append(' '.join([token_seq[g] for g in group]))

In [ ]:
new_token_seq = deepcopy(token_seq)

for i, group in enumerate(groups):
    new_token_seq[group[0]] = links[i]
    if len(group) > 1:
        for g in group[1:]:
            new_token_seq[g] = ''

In [ ]:
display(HTML(' '.join(new_token_seq)))

we want the shortest version of the most specific thing. precise precision